## Badanie wpływu szumu na proces uczenia 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [3]:
def fun(x,y):
    return np.log10(np.abs(x))*5*np.cos(y)+(0.55*(x+y))

In [4]:
x_n = np.outer(np.linspace(-10, 10, 100), np.ones(100))
y_n = x_n.copy().T
z_n = fun(x_n,y_n)

In [5]:
models = [
    { 'kernel': 'linear', 'C': 64.76855, 'epsilon': 0.006793}, ##linear_mae
    { 'kernel': 'linear', 'C': 21.62282, 'epsilon': 0.01343 }, ##linear_mse
    { 'kernel': 'linear', 'C': 6.09208, 'epsilon': 0.00285},##linear_r2
    { 'kernel': 'rbf', 'C': 7.89739, 'epsilon': 0.020708, 'gamma': 30.10516},#rbf_mae
    { 'kernel': 'rbf', 'C': 7.714452, 'epsilon': 0.01036, 'gamma': 24.57617 }, ##rbf_mse, 
    {  'kernel': 'rbf', 'C': 62.90105, 'epsilon': 0.00019, 'gamma': 21.73123}, ##rbf_r2
    { 'kernel': 'poly', 'C': 98.05169, 'epsilon': 0.01403, 'degree': 6, 'coef0': 9.89688},##poly_mae,
    { 'kernel': 'poly', 'C': 52.47482, 'epsilon': 0.00283, 'degree': 7, 'coef0': 9.45739}, ##poly_mse
    { 'kernel': 'poly', 'C': 0.34617, 'epsilon': 0.411307, 'degree': 4, 'coef0': 2.2253}, ##poly_r2
    { 'kernel': 'sigmoid', 'C': 70.96374, 'epsilon': 1.4122, 'coef0': 0.7384},##sigmoid_mae
    { 'kernel': 'sigmoid', 'C': 89.26535, 'epsilon': 0.05399, 'coef0': 7.50992}, ##sigmoid_mse
    {'kernel': 'sigmoid', 'C': 50.36861, 'epsilon': 1.50263, 'coef0': 0.87763} ##sigmoid_r2
]

In [6]:
noise_levels = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0]
results = []

In [ ]:

for level in noise_levels:

    for params in models: 
  
        noise = np.random.normal(0,level,(100,100))
        noised_z = z_n + noise
        

        Xn = np.column_stack((x_n.flatten(), y_n.flatten()))
        yn = noised_z.flatten()

        Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn, yn, test_size=0.1, random_state=85)

        Xn_scaler = StandardScaler()
        Xn_train = Xn_scaler.fit_transform(Xn_train)
        Xn_test = Xn_scaler.transform(Xn_test)

        yn_scaler = StandardScaler()
        yn_train = yn_scaler.fit_transform(yn_train[:,None])[:,0]
        yn_test = yn_scaler.transform(yn_test[:,None])[:,0]
    
        model = svm.SVR(**params, verbose=2)
        model.fit(Xn_train, yn_train)

        yn_pred_train = model.predict(Xn_train)
        yn_pred_test = model.predict(Xn_test)
        
        r2_train = model.score(Xn_train, yn_train)
        r2_test = model.score(Xn_test, yn_test)
        print(f"R2 na zbiorze trenującym: {r2_train}")
        print(f"R2 na zbiorze testującym: {r2_test}")

        mse_train = metrics.mean_squared_error(yn_train, yn_pred_train)
        mse_test = metrics.mean_squared_error(yn_test, yn_pred_test)

        print(f"MSE na zbiorze trenującym: {mse_train}")
        print(f"MSE na zbiorze testującym: {mse_test}")

        mae_train = metrics.mean_absolute_error(yn_train, yn_pred_train)
        mae_test = metrics.mean_absolute_error(yn_test, yn_pred_test)

        print(f"MAE na zbiorze trenującym: {mae_train}")
        print(f"MAE na zbiorze testującym: {mae_test}")
        results.append({'params': params, 'noise': level, 'r2_train': r2_train, 'r2_test': r2_test, 'mse_train': mse_train, 'mse_test': mse_test, 'mae_test': mae_test, 'mae_train': mae_train})

[LibSVM]

In [ ]:
df = pd.DataFrame(results)
df.to_csv('./noises_results.csv')